# Wine Quality Predictor

The first section of this project will predict using only numpy calculations

I will use Dropout, the Adam Optimizer and and Minibatch Sampling on this Model 

In [104]:
import numpy as np
from sklearn import preprocessing

In [109]:
# load the data from winequality dataset
# has about 5K entries
# data from https://machinelearningmastery.com/standard-machine-learning-datasets/

def loadDataset():
    data = np.genfromtxt("data/winequality-white.csv", delimiter = ";")
    data = np.delete(data, 0, axis = 0)
    
    train, test = data[:3500, :], data[3500:, :]
    
    train_x = train[:,:-1].T
    train_y = train[:,-1].reshape(1, train.shape[0])
    
    train_x = preprocessing.normalize(train_x)
    
    test_x = test[:,:-1].T
    test_y = test[:,-1].reshape(1, test.shape[0])
    
    return (train_x, train_y, test_x, test_y)
    

In [110]:
train_x, train_y, test_x, test_y = loadDataset()
print(train_x)

[[0.01678885 0.01510996 0.0194271  ... 0.01846773 0.0153498  0.01463028]
 [0.01556571 0.01729523 0.01614222 ... 0.02478984 0.01729523 0.01902476]
 [0.01655237 0.01563279 0.01839152 ... 0.0459788  0.01655237 0.01379364]
 ...
 [0.01583671 0.01742038 0.01720922 ... 0.01736759 0.01784269 0.01747317]
 [0.01513307 0.01647823 0.01479678 ... 0.01681452 0.01782339 0.01345161]
 [0.01420032 0.0153299  0.0162981  ... 0.01936408 0.01694357 0.02113912]]
